### No structure notebook;
#### Space for testing & experiments

In [ ]:
# Observed health points at given lvl from own characters:
lvl_to_hp = {
    '1-20': [20,51,90,134,182,234,290,348,410,474,540,609,680,753,828,905,983,1064,1146,1230],
    '21-40':[1315,1402,1490,1580,1671,1764,1858,1953,2050,2148,2247,2344,2442,2551,2649,2754,2854,2973,3081,3190],
    '41-46':[3288,3387,3512,3637,3751,3866],
    '51-60': [4455,4558,4679,4802,4943,5067,5192,5317,5420,5571],
    '70-75': [6887,7022,7159,7278,7416,7554],
    '95': [10480],
    '101': [11401],
    '123-125': [14950, 15090, 15258]
}

# Function to create suitable axes data for seaborn plot
def axes_creator(ob : dict):
    hps = []
    for val in ob.values():
        hps.extend(val)
        
    lvls = []
    for x in ob.keys():
        lvl_arr = x.split("-")
        if len(lvl_arr) == 1:
            lvls.append(int(lvl_arr[0]))
        else:
            for i in range(int(lvl_arr[0]), int(lvl_arr[1]) + 1):
                lvls.append(int(i))
    return lvls, hps
    
x, y = axes_creator(lvl_to_hp)

In [ ]:
import pandas as pd
import seaborn as sns

# Seaborn plot
df = pd.DataFrame({'lvl': x, 'hp': y})
sns.relplot(data=df, x='lvl', y='hp')

In [ ]:
# Testing data classes from game_structures
from game_structures import *

test_item = Item(30,'pw','ar','armor plate test name', 
    {"armor": 50, "health_points": 100, "strength": 40, "speed_attack" : 36, 'dodge': 10}
)
test_item.__repr__()
test_eq = Eq()

test_eq.add(test_item)
print(test_eq.items," -> new item in eq object")
test_eq.add(Item(40,'pt','he','helmet test n.',
    {"armor": 25, "hp_regen": 74, "agility": 111, "attack_speed": 0.24}
))
print(test_eq.items," -> 2nd item in eq object")
test_eq.add(Item(35,'pwb','ar','front armor plate',
    {"armor": 56, "health_points": 142, "hp_regen": 64, "strength": 34, "attack_speed": 0.28}
))
print(test_eq.items," -> new added item replaced previous armor")
# check summed up statistics
print(test_eq.stats)

# Player object testing

test_char = Character(100, 'm')
test_char.__repr__()
print(test_char.stats)

Character.get_stats(50, 'p', test_eq.stats)



In [ ]:
import os
import csv

def csv_creator(lvl : int, filename : str):
    item_folder = os.listdir('items/')
    for entry in item_folder:
        if entry.endswith('.csv'):
            with open('items/' + entry, encoding = 'utf-8') as fr:
                try:
                    csv_reader = csv.reader(fr)
                    csv_reader.__next__()
                    item_lines = []
                    for line in csv_reader:
                        #print(line)
                        #lvl_chars = str(lvl)
                        if lvl < 100:
                            if int(line[0][:2]) < lvl:
                                continue
                            elif int(line[0][:2]) == lvl:
                                item_lines.append(line)
                            else:
                                break
                        else:
                            if int(line[0][:3]) < lvl:
                                continue
                            elif int(line[0][:3]) == lvl:
                                item_lines.append(line)
                            else:
                                break
                except StopIteration:
                    print(StopIteration)
                finally:
                    with open(filename, 'a', encoding = 'utf-8', newline='') as fw:
                        csv_writer = csv.writer(fw)
                        csv_writer.writerows(item_lines)

csv_creator(101, "items/_selected_items.csv")

In [1]:
from pprint import pprint
from game_structures import *
import ast

# Loading characters (eq from csv file)
sets_101 = {
    't103': ('Skrzący łęczar burzy', 'Strzały smoczego jeźdźca VI', 'Pustynna kolczuga mistrzów V', 'Maska oszalałej wdowy', 'Buty do tanga', 'Skinięcie Yin', 'Pieczęć siedmiu wynaturzeń', 'Symbol Wszelkich Przekształceń'),
    'm102': ('Insygnium uwolnienia burzy VIII', 'Atrament wykładowcy', 'Kapa bagiennej aury', 'Kapelusz magicznej kapituły', 'Azurytowe cholewy nieumarłych', 'Rękawice kochanka Nocnej Kochanki', 'Pierścień rybiego oka', 'Klątwa leśnej niezapominajki'),
    'p102': ('Skalpownik', 'Skorupa nieszczęścia VIII', 'Kurta światłego generała', 'Rogata żelazna przyłbica', 'Torfowa jurność stróża', 'Pamięć porażki', 'Zaręczynowy pierścionek arystokratki', 'Serce stepowe'),
    'w101': ('Boleść złamanego serca', 'Tarcza cmentarnych lilii', 'Smoczy pancerz mistrzów V', 'Korona Margorii', 'Trybut ognistego bóstwa II', 'Łapy Ośmiu Trygramów', 'Oczko Duvy', 'Emanująca duma urgrapów'),
    'h103': ('Bezszelestny łuk uciszania', 'Struny Deprawacji', 'Wibrująca ochrona pohańbionych', 'Czaszka Naruona', 'Żaroodporne podeszwy', 'Skinięcie Yang', 'Obrączka Różanity', 'Zaklęte padło jaskini'),
    'b101': ('Skażony kindżał bandziora', 'Rozerwimord VIII', 'Karacena toskycznego ukąszenia', 'Maska porażającej niegodziwości', 'Płetwobuty', 'Zmutowane łapy królikoluda', 'Pradawny pierścień zniszczenia', 'Złota łza słońca')
}

def load_characters(chardata: dict, filename: str):
    chars = {}
    for key, val in chardata.items():
        eq = Eq()
        for x in val:
            with open('items/' + filename, encoding = 'utf-8') as fr:
                for _ in range(888):
                    row_str = fr.readline().split(',', 4)
                    try:
                        if x == row_str[3]:
                            x_row = '"'.join(ast.literal_eval(row_str[4])).replace('"','')
                            new_item = Item(int(row_str[0]),row_str[1],row_str[2],row_str[3],eval(x_row))
                            eq.add(new_item)
                            break
                    except IndexError:
                        print(row_str)
        new_char = Character(int(key[1:]), str(key[0]), eq.stats, eq.bonuses)
        chars[f'{new_char.proffesion}{new_char.lvl}'] = new_char
    return chars

# Parse stats dict values into ints
# Counts base params x2 (bug)
chars = load_characters(sets_101, "_selected_items.csv")

# for x in sets_101:
#     print(f'{chars[x]} -> {chars[x].stats()}')
F_Sim = FightSimulator([chars['t103'], chars['p102']])
F_Sim.starting_stats(102)
F_Sim.starting_stats(103)
print(F_Sim._stats_types[0])
print(F_Sim._stats_types[1])

TypeError: 'Field' object does not support item assignment

In [2]:
from game_structures import *

man_eq = Eq()

items_stats = [
{'crit_strike': '1', 'physical_dmg': '938-1146', 'strength': '60', 'agility': '60', 'energy': '17', 'dodge': '10', 'attack_speed': '33', 'wound_dmg': '17,345'},
{'armor': '449', 'poison_resist': '5', 'block': '126', 'physical_crit': '6', 'energy': '17', 'health_points': '551', 'dodge_reduction': '10', 'pierce_block': '60', 'light_resist': '2', 'attack_speed': '33'},
{'armor': '689', 'crit_strike': '1', 'strength': '60', 'energy': '17', 'health_points': '616', 'strength_hp': '1', 'dodge_reduction': '10', 'attack_speed': '58'},
{'armor': '218', 'armor_reduction': '7', 'crit_strike': '2', 'physical_crit': '12', 'all_features': '32', 'strength': '57', 'energy': '16', 'health_points': '403', 'legendary_bonus': 'lastheal,95', 'dodge_reduction': '10', 'fire_resist': '-3', 'frost_resist': '3', 'attack_speed': '56'},
{'armor': '179', 'strength': '154', 'crit_reduction': '2', 'dodge_reduction': '9', 'attack_speed': '53', 'as_reduction': '34'},
{'armor': '154', 'crit_strike': '3', 'physical_crit': '6', 'strength': '54', 'energy': '16', 'health_points': '702', 'legendary_bonus': 'lastheal,90', 'dodge_reduction': '18', 'attack_speed': '53'},
{'armor_reduction': '6', 'poison_resist': '10', 'crit_strike': '3', 'all_features': '48', 'energy': '15', 'hp_regen': '270', 'health_points': '930', 'legendary_bonus': 'dmgred,80', 'crit_reduction': '2', 'mana': '20', 'light_resist': '-6', 'attack_speed': '68', 'as_reduction': '31'},
{'poison_resist': '5', 'crit_strike': '2', 'physical_crit': '12', 'strength': '114', 'agility': '59', 'energy': '33', 'dodge': '10', 'health_points': '1191', 'legendary_bonus': 'dmgred,99', 'dodge_reduction': '10', 'attack_speed': '107'}
]

acc_letters = ['1h', 'sh', 'ar', 'he', 'bo', 'gl', 'ri', 'ne']

for i in range(len(items_stats)):
    new_item = Item(100, 'w', acc_letters[i], acc_letters[i],items_stats[i])
    #print(new_item.stats)
    man_eq.add(new_item)

# print('--- EQ --- info ---')
# print(man_eq.stats)
# print(man_eq.items)
# print(man_eq.bonuses)
man_char = Character(101, 'w', man_eq.stats, man_eq.bonuses)
print('--- CHAR --- info ---')
print(man_char.eq_stats)
print(man_char.stats())
print(man_char.bonuses)

--- CHAR --- info ---
{'crit_strike': 12, 'physical_dmg': 1042.0, 'strength': 499, 'agility': 119, 'energy': 131, 'dodge': 20, 'attack_speed': 461, 'wound_dmg': 345, 'wound_chance': 17, 'armor': 1689, 'poison_resist': 20, 'block': 126, 'physical_crit': 36, 'health_points': 4393, 'dodge_reduction': 67, 'pierce_block': 60, 'light_resist': -4, 'strength_hp': 1, 'armor_reduction': 13, 'all_features': 80, 'fire_resist': -3, 'frost_resist': 3, 'crit_reduction': 4, 'as_reduction': 65, 'hp_regen': 270, 'mana': 20}
{'crit_strike': 15.02, 'physical_dmg': 1042.0, 'strength': 1064, 'agility': 221, 'energy': 131, 'dodge': 27, 'attack_speed': 785, 'wound_dmg': 345, 'wound_chance': 17, 'armor': 1689, 'poison_resist': 20, 'block': 126, 'physical_crit': 177, 'health_points': 22178, 'dodge_reduction': 67, 'pierce_block': 60, 'light_resist': -4, 'strength_hp': 1, 'armor_reduction': 13, 'fire_resist': -3, 'frost_resist': 3, 'crit_reduction': 4, 'as_reduction': 65, 'hp_regen': 270, 'mana': 20, 'intellect':